In [ ]:
import torch as th
from active_critic.utils.gym_utils import make_policy_dict
from stable_baselines3.common.torch_layers import CombinedExtractor
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import numpy as np
from gym import Env
from active_critic.policy.active_critic_policy import ActiveCriticPolicy
import numpy as np
import torch as th
from metaworld.envs import \
    ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from metaworld.policies import *
from gym.wrappers import TimeLimit
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from active_critic.utils.rollout import rollout, make_sample_until, flatten_trajectories
from stable_baselines3.common.type_aliases import GymEnv
from gym import Env
import gym
from stable_baselines3.common.evaluation import evaluate_policy
from active_critic.utils.gym_utils import sample_expert_transitions, parse_sampled_transitions, new_epoch_reach, DummyExtractor
from stable_baselines3.common.type_aliases import Schedule

from active_critic.utils.tboard_graphs import TBoardGraphs


def evaluate_policy(policy, env, episodes, device, path, logname, step):
    transitions = sample_expert_transitions(
        policy, env, episodes)
    actions, observations, rewards = parse_sampled_transitions(
        transitions=transitions, new_epoch=new_epoch_reach, extractor=DummyExtractor(), device=device)
    rewards = rewards.unsqueeze(-1)
    last_reward, _ = rewards.max(dim=1)
    success = (last_reward == 1)
    success = success.type(th.float)
    tboard = TBoardGraphs(logname, data_path=path)

    tboard.plotDMPTrajectory(actions[0], actions[0], th.zeros_like(actions[0]),
                                      None, None, None, stepid=step, save=False, name_plot='Trajectory', path=None,
                                      tol_neg=None, tol_pos=None, inpt=None, name='Trajectory', opt_gen_trj=None, window=None)
    tboard.plotDMPTrajectory(rewards[0], rewards[0], th.zeros_like(rewards[0]),
                                      None, None, None, stepid=step, save=False, name_plot='Rewards', path=None,
                                      tol_neg=None, tol_pos=None, inpt=None, name='Rewards', opt_gen_trj=None, window=None)
    tboard.addTrainScalar('rewards', value=last_reward.mean().to('cpu'), stepid=step)      
    tboard.addTrainScalar('success', value=success.mean().to('cpu'), stepid=step)      

In [ ]:
from active_critic.utils.gym_utils import (
    make_vec_env, 
    make_dummy_vec_env, 
    sample_expert_transitions_rollouts, 
    make_pomdp_rollouts, 
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew
)
import gym
from stable_baselines3 import PPO
import torch
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv
from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.evaluation import evaluate_policy
import warnings
from typing import Any, Dict, Optional, Type, Union

import numpy as np
import torch as th
from gym import spaces
from torch.nn import functional as F

from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from stable_baselines3.common.policies import ActorCriticCnnPolicy, ActorCriticPolicy, BasePolicy, MultiInputActorCriticPolicy
from stable_baselines3.common.type_aliases import GymEnv, MaybeCallback, Schedule
from stable_baselines3.common.utils import explained_variance,  get_schedule_fn

from torch.utils.data import DataLoader
from imitation.algorithms.adversarial import gail 
from imitation.util.networks import RunningNorm
from imitation.rewards.reward_nets import BasicRewardNet
from stable_baselines3.ppo import MlpPolicy
from active_critic.utils.tboard_graphs import TBoardGraphs
from active_critic.model_src.transformer import PositionalEncoding

import copy

from active_critic.utils.gym_utils import make_policy_dict, ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE, ResetCounterWrapper, TimeLimit, StrictSeqLenWrapper, ImitationLearningWrapper
from active_critic.TQC.tqc import TQC
from active_critic.TQC.tqc_policy import TQCPolicyEval

def run_experiment(device):
    pass
device='cuda'
lookup_freq = 1
env, vec_expert = make_dummy_vec_env(name='pickplace', seq_len=200)
val_env, _ = make_dummy_vec_env(name='pickplace', seq_len=200)

transitions, rollouts = sample_expert_transitions_rollouts(vec_expert.predict, env, 10)
env.envs[0].reset_count = 0
pomdp_rollouts = make_pomdp_rollouts(rollouts, lookup_frq=lookup_freq, count_dim=10)
pomdp_transitions = rollout.flatten_trajectories(pomdp_rollouts)

pomdp_env, pomdp_vec_expert = make_dummy_vec_env_pomdp(name='pickplace', seq_len=200, lookup_freq=lookup_freq)
policy_kwargs = {'net_arch' : [32, 32, 32]}
tqc_learner = TQC(policy='MlpPolicy', env=pomdp_env, device=device, policy_kwargs=policy_kwargs)


    

In [ ]:
def get_avr_succ_rew_det(env, learner, epsiodes):
    success = []
    rews = []
    for i in range(epsiodes):
        obs = env.reset()
        done = False
        while not done:
            action, _ = learner.predict(obs, deterministic=True)
            obs, rew, done, info = env.step(action)
            rews.append(rew)
            if info[0]['success'] > 0:
                success.append(info[0]['success'])
                break
            if done:
                success.append(0)
    return np.array(success), np.array(rews)

In [ ]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=pomdp_transitions,
    device=device,
    policy=tqc_learner.policy)

tboard = TBoardGraphs(logname='BC TQC pickplace lookup netarch[32 32 32] '+str(lookup_freq) , data_path='/data/bing/hendrik/gboard/')
for i in range(10000):
    bc_trainer.train(n_epochs=20)
    success, rews = get_avr_succ_rew_det(env=pomdp_env, learner=bc_trainer.policy, epsiodes=200)
    tboard.addTrainScalar('Reward', value=th.tensor(rews.mean()), stepid=i)
    tboard.addTrainScalar('Success Rate', value=th.tensor(success.mean()), stepid=i)

In [ ]:
success, rews = get_avr_succ_rew_det(env=pomdp_env, learner=tqc_learner.policy, epsiodes=200)

In [ ]:
success.mean()

In [ ]:
rews.mean()

In [ ]:
tboard = TBoardGraphs(logname='BC TQC pickplace continue 17.5 lookup '+str(lookup_freq) , data_path='/data/bing/hendrik/gboard/')


In [ ]:
pomdp_env_eval, _ = make_dummy_vec_env_pomdp(name='pickplace', seq_len=200, lookup_freq=lookup_freq)


In [ ]:
for i in range(10000):
    tqc_learner.learn(total_timesteps=200, log_interval=100000)    
    success, rews = get_avr_succ_rew(env=pomdp_env_eval, learner=tqc_learner.policy, epsiodes=200)
    tboard.addTrainScalar('Reward', value=th.tensor(rews.mean()), stepid=i)
    tboard.addTrainScalar('Success Rate', value=th.tensor(success.mean()), stepid=i)
    print(f'succes: {success.mean()}')
    print(f'rews: {rews.mean()}')
    print(f'resets: {pomdp_env.envs[0].reset_count}')

In [ ]:
obs = pomdp_env.reset()

In [ ]:
obs

In [ ]:
obs

In [ ]:
pomdp_env.envs[0].reset_count

In [ ]:
done = False
while not done:
    action = np.array([[0,0,0,0]])
    obs, rew, done, info = pomdp_env.step(action)

In [ ]:
seq_env, expert = make_dummy_vec_env(name='reach', seq_len=100)
seq_env_eval, expert = make_dummy_vec_env(name='reach', seq_len=100)

policy_kwargs = dict(net_arch=[64, 64], n_critics=1)
tqc_learner = TQC(policy='MlpPolicy',learning_rate=0.001, env=seq_env, top_quantiles_to_drop_per_net=2, verbose=1, policy_kwargs=policy_kwargs)
path = '/home/hendrik/Documents/master_project/LokalData/baselines'

In [ ]:
obs = seq_env.reset()
obs = th.tensor(obs).unsqueeze(0)
action = th.tensor(np.array([0,0,0,0]), dtype=th.float).unsqueeze(0)
tqc_learner.policy.evaluate_actions(obs=obs, actions=action)

In [ ]:
for i in range(100):
    tqc_learner.learn(2000, log_interval=float('inf'))
    evaluate_policy(policy=tqc_learner.policy, env=seq_env_eval, episodes=100, device='cuda', path=path, logname='tqc_reach_success_stop', step=seq_env.envs[0].reset_count)


In [ ]:
d

In [ ]:
e = np.array(d)

In [ ]:
e

In [ ]:
seq_env.envs[0].reset_count

In [ ]:
import torch as th

In [ ]:
obsv = seq_env.reset()
rews = []
success = []
actions = []
for i in range(10000):
    action = tqc_learner.policy.predict(obsv[0])
    actions.append(action[0])
    obsv, rew, done, info = seq_env.step(action)
    rews.append(rew)
    if done[0]:
        success.append(info[0]['success'])

In [ ]:
actions = np.array(actions)
actions.shape

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1)
ax.plot(np.arange(len(rews)), actions[:,3])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1)
ax.plot(np.arange(len(rews)), np.array(rews))

In [ ]:
success